<a href="https://colab.research.google.com/github/surya323-ma/skillcraft-technology/blob/main/hand_gesture_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install tensorflow opencv-python


In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gti-upm/leapgestrecog")

print("Path to dataset files:", path)


100%|██████████| 2.13G/2.13G [00:29<00:00, 76.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1


In [17]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the path to the dataset
dataset_path = path  # Use the path obtained from the previous step

# Update the dataset path to point to the directory containing gesture subfolders
gesture_data_path = os.path.join(dataset_path, 'leapGestRecog')

# Load images and labels
images = []
labels = []

# Assuming the dataset has a structure where images are in folders named after their labels
for gesture_label in os.listdir(gesture_data_path):
    gesture_path = os.path.join(gesture_data_path, gesture_label)
    if os.path.isdir(gesture_path):
        # Now iterate through the subject subfolders within each gesture folder
        for subject_folder in os.listdir(gesture_path):
            subject_path = os.path.join(gesture_path, subject_folder)
            if os.path.isdir(subject_path):
                for img_file in os.listdir(subject_path):
                    img_path = os.path.join(subject_path, img_file)
                    img = cv2.imread(img_path)
                    if img is not None:  # Check if the image was loaded successfully
                        img = cv2.resize(img, (64, 64))  # Resize to 64x64
                        images.append(img)
                        labels.append(gesture_label) # Use the gesture folder name as the label
                    else:
                        print(f"Warning: Could not load image {img_path}")


# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize pixel values
images = images.astype('float32') / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [18]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert to categorical
num_classes = len(label_encoder.classes_)
y_train_categorical = to_categorical(y_train_encoded, num_classes)
y_test_categorical = to_categorical(y_test_encoded, num_classes)


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Train the model
history = model.fit(X_train, y_train_categorical, epochs=20, batch_size=32, validation_data=(X_test, y_test_categorical))


Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 103s 202ms/step - accuracy: 0.6615 - loss: 0.9603 - val_accuracy: 0.9910 - val_loss: 0.0206
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 199ms/step - accuracy: 0.9747 - loss: 0.0625 - val_accuracy: 0.9890 - val_loss: 0.0198
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 197ms/step - accuracy: 0.9826 - loss: 0.0403 - val_accuracy: 0.9918 - val_loss: 0.0143
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 183ms/step - accuracy: 0.9854 - loss: 0.0322 - val_accuracy: 0.9905 - val_loss: 0.0134
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 90s 180ms/step - accuracy: 0.9867 - loss: 0.0276 - val_accuracy: 0.9912 - val_loss: 0.0132
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 90s 180ms/step - accuracy: 0.9884 - loss: 0.0221 - val_accuracy: 0.9915 - val_loss: 0.0143
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 148s 192ms/step - accuracy: 0.9879 - loss: 0.0237 - val_accuracy: 0.9905 - val_loss: 0.0136
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 189ms/step - accuracy: 0.9875 - l

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy: {test_accuracy:.2f}')


  3/125 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 1.0000 - loss: 0.0057    

In [ ]:
model.save('hand_gesture_recognition_model.h5')
